#  Setting Up an Apache Web Server

A web server allows for data hosted on one computer (the "server") to be accessable by other nodes on the network. We will refer to any nodes wanting to access the data as "clients".

This example notebook will demonstrate how to set up a client-server network topology on FABRIC, install Apache on a server node, and download files from the server node to a single client node.


## Configure the Environment

## Setup the Experiment

#### Import FABRIC API

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Step 4: Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated WAN Ethernet, as outlined in the [Create a Wide Area Ethernet (Layer 2) tutorial](../../fablib_api/create_l2network_wide_area_auto/create_l2network_wide_area_auto.ipynb). More advanced topologies can be created, so long as the server node is routable by each client attempting to access the server's data.

In [ ]:
slice_name = 'MySlice-apache-web-server'
[site1,site2]  = fablib.get_random_sites(count=2)
print(f"Sites: {site1}, {site2}")

server_name = 'server'
client_name = 'client'
network_name='net1'
server_nic_name = 'server_nic'
client_nic_name = 'client_nic'

# We will use Ubuntu 20.04 for both nodes
image = 'default_ubuntu_20'

In this example, we will set up an L2 network. As discuss above, a different network configuration, such as using the FABNet L3 network should work, so long as the clients and server can route to each other.

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Network
    net1 = slice.add_l2network(name=network_name, subnet=IPv4Network("192.168.1.0/24"))

    # Node1
    server = slice.add_node(name=server_name, site=site1, image=image)
    server_iface = server.add_component(model='NIC_Basic', name=server_nic_name).get_interfaces()[0]
    server_iface.set_mode('auto')
    net1.add_interface(server_iface)
    
    # Node2
    client = slice.add_node(name=client_name, site=site2, image=image)
    client_iface = client.add_component(model='NIC_Basic', name=client_nic_name).get_interfaces()[0]
    client_iface.set_mode('auto')
    net1.add_interface(client_iface)    

    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Get Slice information

In [ ]:
slice = fablib.get_slice(name=slice_name)
server = slice.get_node(name=server_name)
client = slice.get_node(name=client_name)

## Install Apache on the Server Node

In [ ]:
apache_root_dir = "/var/www/html" # Default Root Directory to Store Web Server Files

try:
    print("Installing Apache...")
    stdout, stderr = server.execute("sudo apt-get update && sudo apt-get install -y apache2")
    print("Finished installing Apache.")
    
except Exception as e:
    print(f"Exception: {e}")

## Optional: Set Up Firewall

In [ ]:
commands = [
    "sudo ufw allow 'Apache'",                                # Allow Port 80 Through Firewall
    "sudo ufw allow ssh",                                     # Allow Port 22 Through Firewall (IMPORTANT)
    "echo 'y' | sudo ufw enable"                              # Enable the Firewall
]

try:
    print("Setting up firewall...")
    for command in commands:
        stdout, stderr = server.execute(command)
    print("Finished setting up firewall.")
    
except Exception as e:
    print(f"Exception: {e}")

## Check to Ensure the Firewall is Active

The command below should result in a table that looks like:

```
Status: active

To                         Action      From
--                         ------      ----
Apache                     ALLOW       Anywhere                  
22/tcp                     ALLOW       Anywhere                  
Apache (v6)                ALLOW       Anywhere (v6)             
22/tcp (v6)                ALLOW       Anywhere (v6)             
```

In [ ]:
try:
    stdout, stderr = server.execute('sudo ufw status')
    print(stdout)
    
except Exception as e:
    print(f"Exception: {e}")

## Check to Ensure the Apache Service is Active

The command below should result in an entry that looks like:

```
● apache2.service - The Apache HTTP Server
     Loaded: loaded (/lib/systemd/system/apache2.service; enabled; vendor preset: enabled)
     Active: active (running) since ...
```

In [ ]:
try:
    stdout, stderr = server.execute('sudo systemctl status apache2')
    print(stdout)
    
except Exception as e:
    print(f"Exception: {e}")

## Optional: Change Directory of Where Apache Files are Stored

Update the `apache_root_dir` variable below to the directory you would like to store your files.

In [ ]:
apache_root_dir = "/home/ubuntu/apache_contents"
change_root = f"mkdir -p {apache_root_dir} && sudo sed -i 's,/var/www/html,{apache_root_dir},g' /etc/apache2/sites-available/000-default.conf && sudo sed -i 's,/var/www/html,{apache_root_dir},g' /etc/apache2/sites-available/default-ssl.conf && sudo sed -i 's,/var/www/,{apache_root_dir},g' /etc/apache2/apache2.conf"

try:
    stdout, stderr = server.execute(change_root)
    print(stderr)
    
except Exception as e:
    print(f"Exception: {e}")

Restart the Apache service to have the changes take effect.

In [ ]:
try:
    stdout, stderr = server.execute("sudo service apache2 restart")
    print(stderr)
    
except Exception as e:
    print(f"Exception: {e}")

## Save Your Files to the Server

Here, we will create a new file called `download_me`, which will be a simple text file. Let's write `Congratualtions! The Apache server was setup correctly.` to our file by redirecting the string into the command `tee`, which writes the string into the file.

In [ ]:
create_file = f'echo "Congratualtions! The Apache server was setup correctly." | sudo tee {apache_root_dir}/download_me'
try:
    stdout, stderr = server.execute(create_file)
    print("The file was created successfully.")
    
except Exception as e:
    print(f"Exception: {e}")

## Download the File to the Client

We are now ready to test our web server! Let's download the `download_me` file we created to the client node using `wget`. Apache uses port 80 for normal, unencrypted web traffic.

In [ ]:
server_addr = server.get_interface(network_name=network_name).get_ip_addr()


In [ ]:
try:
    stdout, stderr = client.execute(f'wget {server_addr}:80/download_me')
    print(stdout,stderr)
    stdout, stderr = client.execute(f'cat download_me')
    print(stdout)
    
except Exception as e:
    print(f"Exception: {e}")

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")